# Forward and Inverse Kinematics (Analytical Method)
4-DOF Robotic Arm using Raspberry Pi Pico

## Import Libraries and Configuration

In [ ]:
import math
from sympy import symbols, cos, sin, Matrix, pi, atan2, sqrt, acos
import numpy as np
import serial
import time

# Pico communication
PICO_PORT = 'COM15'
PICO_BAUD_RATE = 115200

## Arm Kinematics Configuration

In [ ]:
# Robot DH Parameters (4-DOF RRRR Configuration)
l1 = 96.000   # Base height (mm)
l2 = 13.90    # Base offset (mm)
l3 = 150.0    # Link 1 length (mm)
l4 = 143.24   # Link 2 length (mm)
l5 = 127.42   # End-effector length (mm)

print(f"Robot Configuration:")
print(f"l1 (base height): {l1} mm")
print(f"l2 (base offset): {l2} mm")
print(f"l3 (link 1): {l3} mm")
print(f"l4 (link 2): {l4} mm")
print(f"l5 (end-effector): {l5} mm")

## Forward Kinematics (Analytical - Homogeneous Transformation Matrices)

In [ ]:
def forward_kinematics_analytical(q1_deg, q2_deg, q3_deg, q4_deg):
    """Forward kinematics using transformation matrices"""
    # Convert to radians
    Q1 = math.radians(q1_deg)
    Q2 = math.radians(q2_deg)
    Q3 = math.radians(q3_deg)
    Q4 = math.radians(q4_deg)
    
    # Transformation matrix from base to link 1
    A0_1 = np.array([
        [math.cos(Q1), 0, math.sin(Q1), l2 * math.cos(Q1)],
        [math.sin(Q1), 0, -math.cos(Q1), l2 * math.sin(Q1)],
        [0, 1, 0, l1],
        [0, 0, 0, 1]
    ])
    
    # Transformation matrices for each joint
    A1_2 = np.array([
        [math.cos(Q2), -math.sin(Q2), 0, l3 * math.cos(Q2)],
        [math.sin(Q2), math.cos(Q2), 0, l3 * math.sin(Q2)],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    
    A2_3 = np.array([
        [math.cos(Q3), -math.sin(Q3), 0, l4 * math.cos(Q3)],
        [math.sin(Q3), math.cos(Q3), 0, l4 * math.sin(Q3)],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    
    A3_4 = np.array([
        [math.cos(Q4), -math.sin(Q4), 0, l5 * math.cos(Q4)],
        [math.sin(Q4), math.cos(Q4), 0, l5 * math.sin(Q4)],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    
    # Total transformation matrix
    T0_4 = A0_1 @ A1_2 @ A2_3 @ A3_4
    
    # Extract end-effector position
    x = T0_4[0, 3]
    y = T0_4[1, 3]
    z = T0_4[2, 3]
    
    return x, y, z

# Example: Test forward kinematics
q1, q2, q3, q4 = 90, 51.6851, -54.194, -87.49
x, y, z = forward_kinematics_analytical(q1, q2, q3, q4)
print(f"\nForward Kinematics Test:")
print(f"Q1: {q1}°, Q2: {q2}°, Q3: {q3}°, Q4: {q4}°")
print(f"Position: X: {x:.2f} mm, Y: {y:.2f} mm, Z: {z:.2f} mm")

## Inverse Kinematics (Analytical Method)

In [ ]:
def inverse_kinematics_analytical(px, py, pz):
    """Inverse kinematics using analytical method"""
    try:
        # Calculate Q1
        q1 = math.atan2(py, px)
        q1_degrees = math.degrees(q1)
        
        # Working height
        pz_work = pz - l1
        
        # Horizontal distance
        r = math.sqrt(px**2 + py**2)
        
        # Distance to end-effector
        d = math.sqrt(r**2 + pz_work**2)
        
        # Check if position is reachable
        reach = l3 + l4 + l5
        if d > reach:
            print(f"Position unreachable (distance: {d:.2f}, reach: {reach:.2f})")
            return None
        
        # Law of cosines for Q3
        cos_q3 = (d**2 - l3**2 - (l4 + l5)**2) / (2 * l3 * (l4 + l5))
        
        if abs(cos_q3) > 1:
            print("Position unreachable (inverse cosine out of range)")
            return None
        
        q3_rad = math.acos(cos_q3)
        q3_degrees = math.degrees(q3_rad) - 180
        
        # Calculate Q2
        alpha = math.atan2(pz_work, r)
        l34_total = math.sqrt(l3**2 + (l4 + l5)**2 + 2 * l3 * (l4 + l5) * math.cos(q3_rad))
        beta = math.atan2((l4 + l5) * math.sin(q3_rad), l3 + (l4 + l5) * math.cos(q3_rad))
        q2_degrees = math.degrees(alpha) + math.degrees(beta)
        
        # Calculate Q4
        q4_degrees = -q2_degrees - q3_degrees
        
        return q1_degrees, q2_degrees, q3_degrees, q4_degrees
    
    except Exception as e:
        print(f"Inverse kinematics calculation error: {e}")
        return None

# Example: Test inverse kinematics
px, py, pz = 1.53e-14, 250.002605553586, 80.0020053361393
angles = inverse_kinematics_analytical(px, py, pz)
if angles:
    q1, q2, q3, q4 = angles
    print(f"\nInverse Kinematics Test:")
    print(f"Position: X: {px:.4e} mm, Y: {py:.2f} mm, Z: {pz:.2f} mm")
    print(f"Q1: {q1:.2f}°, Q2: {q2:.2f}°, Q3: {q3:.2f}°, Q4: {q4:.2f}°")

## Send Angles to Raspberry Pi Pico

In [ ]:
def send_angles_to_pico(q1, q2, q3, q4):
    """Send calculated angles to Raspberry Pi Pico"""
    try:
        pico = serial.Serial(PICO_PORT, PICO_BAUD_RATE, timeout=1)
        time.sleep(2)  # Wait for Pico to initialize
        
        # Format message
        message = f"MOVE {q1:.2f} {q2:.2f} {q3:.2f} {q4:.2f}\n"
        
        # Send to Pico
        pico.write(message.encode())
        print(f"Sent to Pico: {message.strip()}")
        
        # Wait for response
        response = pico.readline().decode().strip()
        if response:
            print(f"Pico response: {response}")
        
        pico.close()
        return True
    
    except Exception as e:
        print(f"Error communicating with Pico: {e}")
        return False

## Interactive Kinematics Solver

In [ ]:
def interactive_ik_solver():
    """Interactive inverse kinematics solver"""
    print("\n" + "="*50)
    print("Analytical Inverse Kinematics Solver (Pico)")
    print("="*50)
    
    try:
        px = float(input("Enter X position (mm): "))
        py = float(input("Enter Y position (mm): "))
        pz = float(input("Enter Z position (mm): "))
        
        angles = inverse_kinematics_analytical(px, py, pz)
        
        if angles:
            q1, q2, q3, q4 = angles
            print(f"\nCalculated Angles:")
            print(f"Q1: {q1:.2f}°")
            print(f"Q2: {q2:.2f}°")
            print(f"Q3: {q3:.2f}°")
            print(f"Q4: {q4:.2f}°")
            
            # Option to send to Pico
            send = input("\nSend to Pico? (y/n): ")
            if send.lower() == 'y':
                send_angles_to_pico(q1, q2, q3, q4)
    
    except ValueError:
        print("Invalid input. Please enter numbers.")

# Run interactive solver
interactive_ik_solver()